In [1]:
import numpy as np
import pandas as pd
import eurostat
import pycountry
import seaborn as sns

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
#toc_df = eurostat.get_toc_df()
#toc_df
#info = eurostat.subset_toc_df(toc_df, 'demo_r_mwk_10')
#info
df = eurostat.get_data_df('demo_r_mwk_20')
df.columns = df.columns.str.replace('\\', '_',regex=False)
df = df[df.age != 'UNK']
df

,age,sex,unit,geo_time,2021W50,2021W49,2021W48,2021W47,2021W46,2021W45,...,2000W10,2000W09,2000W08,2000W07,2000W06,2000W05,2000W04,2000W03,2000W02,2000W01
0,TOTAL,F,NR,AD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TOTAL,F,NR,AL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TOTAL,F,NR,AM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TOTAL,F,NR,AT,NaN,NaN,NaN,NaN,1040.0,1042.0,...,834.0,874.0,908.0,914.0,967.0,1071.0,1076.0,1141.0,1062.0,1053.0
4,TOTAL,F,NR,BE,NaN,NaN,1296.0,1180.0,1153.0,1154.0,...,1022.0,1059.0,1051.0,1103.0,1147.0,1172.0,1324.0,1369.0,1387.0,1434.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,Y_LT20,T,NR,RS,NaN,NaN,NaN,NaN,NaN,NaN,...,23.0,21.0,18.0,27.0,21.0,23.0,32.0,31.0,33.0,31.0
766,Y_LT20,T,NR,SE,NaN,5.0,5.0,8.0,11.0,11.0,...,11.0,11.0,12.0,20.0,13.0,10.0,9.0,15.0,11.0,15.0
767,Y_LT20,T,NR,SI,NaN,NaN,NaN,NaN,3.0,0.0,...,3.0,3.0,1.0,4.0,5.0,2.0,3.0,2.0,3.0,6.0
768,Y_LT20,T,NR,SK,NaN,NaN,NaN,6.0,14.0,11.0,...,17.0,16.0,20.0,12.0,23.0,17.0,18.0,19.0,20.0,18.0


In [3]:
uniqueValues = df['age'].unique()
uniqueValues

array(['TOTAL', 'Y20-39', 'Y40-59', 'Y60-79', 'Y_GE80', 'Y_LT20'],
      dtype=object)

In [3]:
df['age'].replace(['Y20-39', 'Y40-59', 'Y60-79', 'Y_GE80', 'Y_LT20'],['20-39', '40-59', '60-79', '80+', '0-20'], inplace=True)
df2=pd.melt(df,id_vars=['age','sex','unit','geo_time'],var_name='yearweek', value_name='deaths')
df2

,age,sex,unit,geo_time,yearweek,deaths
0,TOTAL,F,NR,AD,2021W50,NaN
1,TOTAL,F,NR,AL,2021W50,NaN
2,TOTAL,F,NR,AM,2021W50,NaN
3,TOTAL,F,NR,AT,2021W50,NaN
4,TOTAL,F,NR,BE,2021W50,NaN
...,...,...,...,...,...,...
755209,0-20,T,NR,RS,2000W01,31.0
755210,0-20,T,NR,SE,2000W01,15.0
755211,0-20,T,NR,SI,2000W01,6.0
755212,0-20,T,NR,SK,2000W01,18.0


In [4]:
df_clean = df2.drop(columns = ['unit'])
df_clean[['year','week']] = df_clean.yearweek.str.split("W",expand=True)
df_clean['week'] = df_clean.week.str.extract('(\d+)')
df_clean['year'] = df_clean.year.str.extract('(\d+)')
df_clean['geo_time'] = df_clean['geo_time'].replace("UK", "GB")
df_clean['geo_time'] = df_clean['geo_time'].replace("EL", "GR")
countries = {}
for country in pycountry.countries:
    countries[country.alpha_2] = country.name
df_clean['country'] = [countries.get(country, 'Unknown code') for country in df_clean.geo_time]
df_clean = df_clean[['country','sex','age','yearweek','year','week','deaths']]
#remove this line to get the full time period
df_clean['covid_year']=df_clean['year'] >= '2020'
df_clean.loc[df_clean['covid_year'] == False, 'covid_year'] = '2000-2019 +/- SD'
df_clean.loc[df_clean['covid_year'] == True, 'covid_year'] = df_clean['year']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

df_clean.iloc[0:10]

,country,sex,age,yearweek,year,week,deaths,covid_year
0,Andorra,F,TOTAL,2021W50,2021,50,NaN,2021
1,Albania,F,TOTAL,2021W50,2021,50,NaN,2021
2,Armenia,F,TOTAL,2021W50,2021,50,NaN,2021
3,Austria,F,TOTAL,2021W50,2021,50,NaN,2021
4,Belgium,F,TOTAL,2021W50,2021,50,NaN,2021
5,Bulgaria,F,TOTAL,2021W50,2021,50,NaN,2021
6,Switzerland,F,TOTAL,2021W50,2021,50,NaN,2021
7,Cyprus,F,TOTAL,2021W50,2021,50,NaN,2021
8,Czechia,F,TOTAL,2021W50,2021,50,NaN,2021
9,Germany,F,TOTAL,2021W50,2021,50,NaN,2021


In [ ]:
g = sns.FacetGrid(df_clean.query("sex == 'T'"), col="age", hue="covid_year", row='country', aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('by_country_age.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

In [ ]:
g = sns.FacetGrid(df_clean.query("age == 'TOTAL'"), col="sex", hue="covid_year", row='country', aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('by_country_sex.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

In [9]:
df_total=df_clean.query("age == 'TOTAL' & sex == 'T'").groupby(['yearweek', 'year', 'week', 'covid_year'], as_index=False)['deaths'].sum().reset_index(drop=True)
df_total

,yearweek,year,week,covid_year,deaths
0,2000W01,2000,01,2000-2019 +/- SD,67886.0
1,2000W02,2000,02,2000-2019 +/- SD,68745.0
2,2000W03,2000,03,2000-2019 +/- SD,69668.0
3,2000W04,2000,04,2000-2019 +/- SD,70023.0
4,2000W05,2000,05,2000-2019 +/- SD,68629.0
...,...,...,...,...,...
1141,2021W46,2021,46,2021,83868.0
1142,2021W47,2021,47,2021,71558.0
1143,2021W48,2021,48,2021,66365.0
1144,2021W49,2021,49,2021,43261.0


In [8]:
g = sns.FacetGrid(df_total, hue="covid_year", aspect=2,sharey=False)
g.map(sns.lineplot, 'week', 'deaths', alpha=.7, estimator='mean', ci='sd')
g.set(xlabel="month", ylabel = "deaths per week", xticks=np.arange(1, 53,(53/12) ), xticklabels=months)
g.add_legend(title = '')
for suffix in 'png svg'.split():
    g.savefig('total.'+suffix, dpi=200, bbox_inches='tight', facecolor='white')

SyntaxError: positional argument follows keyword argument (611397274.py, line 1)